# Notebook Setup

In [1]:
if 'google.colab' in str(get_ipython()):
  IN_COLLAB = True
else:
  IN_COLLAB = False

#TODO: CHANGE THIS BASED ON YOUR OWN LOCAL SETTINGS
MY_HOME_ABS_PATH = "/content/drive/MyDrive/W210/co2-flux-hourly-gpp-modeling"

In [2]:
if IN_COLLAB:
  from google.colab import drive
  drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Import Modules

In [3]:
# install required modules quietly
required_packages = ['geopandas', 'pyspark', 'azure-storage-blob']

for p in required_packages: 
  try:
      __import__(p)
  except ImportError:
      %pip install {p} --quiet

In [4]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
import math
import json

from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

import pyspark.pandas as pd
from calendar import monthrange
from datetime import datetime
from io import BytesIO

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# Load locale custome modules
import sys
if IN_COLLAB:
  os.chdir(MY_HOME_ABS_PATH)
  sys.path.insert(0,os.path.abspath("./code/src/tools"))
else:
  sys.path.append(os.path.abspath("./code/src/tools"))

from CloudIO.AzStorageClient import AzStorageClient
from data_pipeline_lib import *

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

## Initialize Spark Session

In [5]:
# Import SparkSession
from pyspark.sql import SparkSession
# Create a Spark Session
spark = SparkSession.builder.master("local[*]").config(
    "spark.jars.packages", 
    "org.apache.hadoop:hadoop-azure:3.3.1,com.microsoft.azure:azure-storage:8.6.6"
    ).getOrCreate()
# Check Spark Session Information
spark

## Define Local File System Constants

In [6]:
root_dir =  MY_HOME_ABS_PATH
tmp_dir =  root_dir + os.sep + '.tmp'
raw_data_dir = tmp_dir
data_dir = root_dir + os.sep + 'data'
cred_dir = root_dir + os.sep + '.cred'
az_cred_file = cred_dir + os.sep + 'azblobcred.json'

if IN_COLLAB:
  raw_data_dir = "/content/drive/MyDrive/CO2_flux_gpp_modeling/DS_capstone_23Spring_CO2/Data/half_hourly_data"

site_metadata_filename = data_dir + os.sep + 'site-metadata.csv'

# Load Train and Test dataset from Azure Storage Blob

In [7]:
# Define target dataset
container = "baseline-data"
ext = "parquet"
ver = "0"
blob_name_base = f"baseline_all_v_{ver}"
train_blob_name_base = f"baseline-train-v-{ver}"
test_blob_name_base = f"baseline-test-v-{ver}"

In [8]:
# Initialized Azure Storage Client
azStorageClient = AzStorageClient(az_cred_file)
sessionkeys = azStorageClient.getSparkSessionKeys()
spark.conf.set(sessionkeys[0],sessionkeys[1])

In [9]:
# Load train dataset
train_blob_path = f"wasbs://{container}@{sessionkeys[2]}.blob.core.windows.net/{train_blob_name_base}"
print(f"Loading train dataset from {train_blob_path}...")
train_df = spark.read.parquet(train_blob_path)

print(f"Data loaded: {train_df.count()} rows x {len(train_df.columns)} columns.")
print("Train data peak:")
train_df.show(5, False)

Loading train dataset from wasbs://baseline-data@mids23spring.blob.core.windows.net/baseline-train-v-0...
Data loaded: 1485926 rows x 37 columns.
Train data peak:
+--------------+------+---------+---------+-------+-----+------+----+-----+---+----+------------------+------------------+------------------+------+------+------+------+------+------+---+----+---------+------+-------+---------+-------+------+----------+-----------+----+----------+-------------+-------------+--------------+---------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|GPP_NT_V

In [10]:
# Load test dataset
test_blob_path = f"wasbs://{container}@{sessionkeys[2]}.blob.core.windows.net/{test_blob_name_base}"
print(f"Loading test dataset from {test_blob_path}...")
test_df = spark.read.parquet(test_blob_path)

print(f"Data loaded: {test_df.count()} rows x {len(test_df.columns)} columns.")
print("Test data peak:")
test_df.show(5, False)

Loading test dataset from wasbs://baseline-data@mids23spring.blob.core.windows.net/baseline-test-v-0...
Data loaded: 559201 rows x 37 columns.
Test data peak:
+--------------+------+---------+---------+-------+-----+-------+----+-----+---+----+------------------+------------------+------------------+------+------+-----+------+------+------+------+----+---------+------+-------+---------+--------+--------+----------+-----------+----+----------+-------------+-------------+--------------+---------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|GPP_NT_VUT_REF|TA_ERA|S

In [11]:
hr_train_df = train_df.filter(col('minute') == 0)
hr_test_df = test_df.filter(col('minute') == 0)

# Randome Forest Regressor (Default)

In [12]:
# Define target variable
target_variable = 'GPP_NT_VUT_REF'

# Train Model
rf = RandomForestRegressor(featuresCol="features", labelCol=target_variable,
                           seed = 42)
model = rf.fit(train_df)

In [13]:
# Evaluate the Model
predictions = model.transform(test_df)

# Compute Evaluation Metrics
# RMSE
evaluator = RegressionEvaluator(labelCol=target_variable, predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE): {rmse:.5}")

# NSE
mean_gpp = predictions.agg(F.avg(target_variable)).first()[0]
nse_formula = 1 - (F.sum((predictions[target_variable] - predictions.prediction)**2) / F.sum((predictions[target_variable] - mean_gpp)**2))
nse = predictions.agg(nse_formula).first()[0]
print(f"Nash-Sutcliffe Efficiency (NSE): {nse:.5}")

Root Mean Squared Error (RMSE): 5.0361
Nash-Sutcliffe Efficiency (NSE): 0.58791


In [14]:
hr_predictions = model.transform(hr_test_df)

# RMSE
evaluator = RegressionEvaluator(labelCol=target_variable, predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(hr_predictions)
print(f"Root Mean Squared Error (RMSE): {rmse:.5}")

# NSE
mean_gpp = hr_predictions.agg(F.avg(target_variable)).first()[0]
nse_formula = 1 - (F.sum((hr_predictions[target_variable] - hr_predictions.prediction)**2) / F.sum((hr_predictions[target_variable] - mean_gpp)**2))
nse = hr_predictions.agg(nse_formula).first()[0]
print(f"Nash-Sutcliffe Efficiency (NSE): {nse:.5}")

Root Mean Squared Error (RMSE): 5.0441
Nash-Sutcliffe Efficiency (NSE): 0.58771


# Model Evaulation Visulation

In [15]:
sites = [x[0] for x in predictions.select("site_id").distinct().collect()]
print(sites)

['ES-LM2', 'US-GLE', 'CA-Cbo', 'FR-Lam', 'US-AR1', 'US-Seg', 'IT-Lsn', 'IT-Cpz', 'US-FR2']


In [20]:
site_predictions = [[x, predictions.filter(F.col('site_id')==x)] for x in sites]

eval_by_site_df = pd.DataFrame(columns=["site_id", "IGBP", "koppen", "rmse", "nse"])

for i, [s, df] in enumerate(site_predictions):

  rmse = evaluator.evaluate(df)

  mean_gpp = df.agg(F.avg(target_variable)).first()[0]
  nse_formula = 1 - (F.sum((df[target_variable] - df.prediction)**2) / F.sum((df[target_variable] - mean_gpp)**2))
  nse = df.agg(nse_formula).first()[0]

  summary_entry = {"site_id":s, "rmse":rmse, "nse":nse, "IGBP":df.head()["IGBP"], "koppen":df.head()["koppen"], }
  eval_by_site_df = eval_by_site_df.append(summary_entry, ignore_index=True)

eval_by_site_df

,site_id,IGBP,koppen,rmse,nse
0,ES-LM2,WSA,Arid,3.58971,0.52358
1,US-GLE,ENF,Cold,3.68021,0.53996
2,CA-Cbo,DBF,Cold,6.61681,0.53284
3,FR-Lam,CRO,Temperate,7.32942,0.52365
4,US-AR1,GRA,Temperate,4.06546,0.34783
5,US-Seg,GRA,Arid,1.44033,0.15944
6,IT-Lsn,OSH,Temperate,3.51599,0.63770
7,IT-Cpz,EBF,Temperate,4.73128,0.46736
8,US-FR2,WSA,Temperate,4.05945,0.51844


In [24]:
eval_by_site_df.sort_values(by=['rmse', 'nse'], ascending=[True,False])

,site_id,IGBP,koppen,rmse,nse
5,US-Seg,GRA,Arid,1.44033,0.15944
6,IT-Lsn,OSH,Temperate,3.51599,0.63770
0,ES-LM2,WSA,Arid,3.58971,0.52358
1,US-GLE,ENF,Cold,3.68021,0.53996
8,US-FR2,WSA,Temperate,4.05945,0.51844
4,US-AR1,GRA,Temperate,4.06546,0.34783
7,IT-Cpz,EBF,Temperate,4.73128,0.46736
2,CA-Cbo,DBF,Cold,6.61681,0.53284
3,FR-Lam,CRO,Temperate,7.32942,0.52365


# Experiment: Use only hourly data

In [21]:
# Define target variable
target_variable = 'GPP_NT_VUT_REF'

# Train Model
rf = RandomForestRegressor(featuresCol="features", labelCol=target_variable,
                           seed = 42)
model = rf.fit(hr_train_df)

# Evaluate the Model
hr_predict = model.transform(hr_test_df)

# Compute Evaluation Metrics
# RMSE
evaluator = RegressionEvaluator(labelCol=target_variable, predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(hr_predict)
print(f"Root Mean Squared Error (RMSE): {rmse:.5}")

# NSE
mean_gpp = hr_predict.agg(F.avg(target_variable)).first()[0]
nse_formula = 1 - (F.sum((hr_predict[target_variable] - hr_predict.prediction)**2) / F.sum((hr_predict[target_variable] - mean_gpp)**2))
nse = hr_predict.agg(nse_formula).first()[0]
print(f"Nash-Sutcliffe Efficiency (NSE): {nse:.5}")

Root Mean Squared Error (RMSE): 5.0951
Nash-Sutcliffe Efficiency (NSE): 0.57932
